In [34]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as spark_sum, date_format
from pyspark.sql.types import DateType
import pandas as pd


In [35]:
# Inicia uma sessão Spark
spark = SparkSession.builder \
    .appName("RelatorioVendasDistribuido") \
    .getOrCreate()

In [36]:
# Lê o arquivo CSV de forma distribuída e particiona por "Country"
df = spark.read.csv("vendas.csv", header=True, inferSchema=True)


In [37]:
# Particiona os dados pelo campo "Country" para otimizar o processamento
df = df.repartition("Country")

In [38]:
# Converte a coluna "Order Date" para o tipo de dado Date
df = df.withColumn("Order Date", col("Order Date").cast(DateType()))

In [39]:
# 1. Identifica o produto mais vendido em termos de quantidade e canal
produto_mais_vendido = df.groupBy("Item Type", "Sales Channel") \
    .agg(spark_sum("Units Sold").alias("Total Units Sold")) \
    .orderBy(col("Total Units Sold").desc()) \
    .limit(1)

In [40]:
# 2. Determina o país e região com o maior volume de vendas (em valor)
vendas_por_pais_regiao = df.groupBy("Country", "Region") \
    .agg(spark_sum("Total Revenue").alias("Total Revenue")) \
    .orderBy(col("Total Revenue").desc()) \
    .limit(1)

In [41]:
# 3. Calcula a média de vendas mensais por produto
df = df.withColumn("Month", date_format(col("Order Date"), "yyyy-MM"))
media_vendas_mensais = df.groupBy("Month", "Item Type") \
    .agg(spark_sum("Units Sold").alias("Total Units Sold"))
media_mensal = media_vendas_mensais.groupBy("Item Type") \
    .agg({"Total Units Sold": "avg"}) \
    .withColumnRenamed("avg(Total Units Sold)", "Average Monthly Sales")

In [42]:
# Função para salvar os resultados em um arquivo Excel
def salvar_em_excel(produto_mais_vendido, vendas_por_pais_regiao, media_mensal, caminho_arquivo):
    # Converte os DataFrames Spark para Pandas
    produto_mais_vendido_pd = produto_mais_vendido.toPandas()
    vendas_por_pais_regiao_pd = vendas_por_pais_regiao.toPandas()
    media_mensal_pd = media_mensal.toPandas()

    # Cria um escritor Excel com múltiplas abas
    with pd.ExcelWriter(caminho_arquivo) as writer:
        produto_mais_vendido_pd.to_excel(writer, sheet_name='Produto Mais Vendido', index=False)
        vendas_por_pais_regiao_pd.to_excel(writer, sheet_name='Vendas Por País e Região', index=False)
        media_mensal_pd.to_excel(writer, sheet_name='Média Vendas Mensais', index=False)



In [43]:
# Define o caminho completo do arquivo Excel
caminho_arquivo_excel = r"C:\Users\josia\OneDrive\Documentos\Script_Ponto_Py\projeto-texte-Bradesco\relatorio_vendas_spark.xlsx"

# Salva os resultados no arquivo Excel
salvar_em_excel(produto_mais_vendido, vendas_por_pais_regiao, media_mensal, caminho_arquivo_excel)

print(f"Relatório gerado com sucesso em '{caminho_arquivo_excel}'")


Relatório gerado com sucesso em 'C:\Users\josia\OneDrive\Documentos\Script_Ponto_Py\projeto-texte-Bradesco\relatorio_vendas_spark.xlsx'
